In [135]:
import pandas as pd
from haversine import haversine, Unit

In [136]:
lm_df = pd.read_csv("data/landmarks_processed.csv")

In [137]:
lm_df

,landmark,station,dir,lat,long,x_pixel,y_pixel,x_res,y_res,intersection,distance
0,white building - southwest,om,s,32.593892,-116.845919,2900,1715,3072,2048,0,0.150093
1,white building - southwest,om,w,32.593892,-116.845919,423,1685,3072,2048,0,0.150093
2,white building - south,om,s,32.594298,-116.844774,1780,1930,3072,2048,0,0.052190
3,communication tower building - east,om,n,32.595487,-116.844141,2545,1725,3072,2048,0,0.095800
4,large bush,om,e,32.599053,-116.839254,318,1420,3072,2048,0,0.698102
...,...,...,...,...,...,...,...,...,...,...,...
117,center tower,wilson,w,34.225460,-118.065367,1790,1195,3072,2048,0,0.626970
118,chara array,wilson,n,34.225874,-118.057203,2805,1480,3072,2048,0,0.202458
119,chara array,wilson,e,34.225874,-118.057203,190,1375,3072,2048,0,0.202458
120,grey building,wilson,n,34.226342,-118.058944,1360,1520,3072,2048,0,0.206730


In [138]:
cam_meta = pd.read_csv("data/cam_meta_rev2.csv")
#cam_meta[['station','dir']] = cam_meta['camera_name'].str.split("-", expand=True)[[0,1]]

In [139]:
cam_meta

,camera_id,camera_name,hpweren_camera_description,cam_lat,cam_long,station,dir,c_lat,c_long,elevation,certainty
0,hpwren0_unknown direction,tje-1-mobo-c,NaN,32.550000,-117.120000,tje,1,0.000000,0.000000,NaN,NaN
1,hpwren1_north,bm-n-mobo-c,Big Black Mountain,33.159927,-116.808092,bm,n,33.173626,-116.811270,4055.0,NaN
2,hpwren1_east,bm-e-mobo-c,Big Black Mountain,33.159927,-116.808092,bm,e,33.158781,-116.790230,4055.0,NaN
3,hpwren1_south,bm-s-mobo-c,Big Black Mountain,33.159927,-116.808092,bm,s,33.157932,-116.807962,4055.0,NaN
4,hpwren1_west,bm-w-mobo-c,Big Black Mountain,33.159927,-116.808092,bm,w,33.159091,-116.858706,4055.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
110,hpwren30_west,marconi-w-mobo-c,Upper Talega,33.519127,-117.480907,marconi,w,33.517974,-117.529087,2300.0,NaN
111,unknown,ws-n-mobo-c,White Star,32.647266,-116.318014,ws,n,32.650875,-116.317954,4000.0,NaN
112,unknown,ws-e-mobo-c,White Star,32.647266,-116.318014,ws,e,32.646405,-116.305941,4000.0,NaN
113,unknown,ws-s-mobo-c,White Star,32.647266,-116.318014,ws,s,32.644742,-116.317985,4000.0,NaN


In [140]:
lm_df = lm_df.merge(cam_meta[['station', 'dir', 'cam_lat', 'cam_long']], left_on=['station', 'dir'], right_on=['station', 'dir'], how='left')

In [141]:
lm_df.head()

,landmark,station,dir,lat,long,x_pixel,y_pixel,x_res,y_res,intersection,distance,cam_lat,cam_long
0,white building - southwest,om,s,32.593892,-116.845919,2900,1715,3072,2048,0,0.150093,32.594762,-116.844694
1,white building - southwest,om,w,32.593892,-116.845919,423,1685,3072,2048,0,0.150093,32.594762,-116.844694
2,white building - south,om,s,32.594298,-116.844774,1780,1930,3072,2048,0,0.052190,32.594762,-116.844694
3,communication tower building - east,om,n,32.595487,-116.844141,2545,1725,3072,2048,0,0.095800,32.594762,-116.844694
4,large bush,om,e,32.599053,-116.839254,318,1420,3072,2048,0,0.698102,32.594762,-116.844694


In [142]:
del lm_df['distance']

In [143]:
lm_df['distance'] = lm_df.apply(lambda x: haversine((x['lat'], x['long']), (x['cam_lat'], x['cam_long'])), axis=1)
del lm_df['cam_lat']
del lm_df['cam_long']

In [144]:
lm_df

,landmark,station,dir,lat,long,x_pixel,y_pixel,x_res,y_res,intersection,distance
0,white building - southwest,om,s,32.593892,-116.845919,2900,1715,3072,2048,0,0.150093
1,white building - southwest,om,w,32.593892,-116.845919,423,1685,3072,2048,0,0.150093
2,white building - south,om,s,32.594298,-116.844774,1780,1930,3072,2048,0,0.052190
3,communication tower building - east,om,n,32.595487,-116.844141,2545,1725,3072,2048,0,0.095800
4,large bush,om,e,32.599053,-116.839254,318,1420,3072,2048,0,0.698102
...,...,...,...,...,...,...,...,...,...,...,...
117,center tower,wilson,w,34.225460,-118.065367,1790,1195,3072,2048,0,0.626970
118,chara array,wilson,n,34.225874,-118.057203,2805,1480,3072,2048,0,0.202458
119,chara array,wilson,e,34.225874,-118.057203,190,1375,3072,2048,0,0.202458
120,grey building,wilson,n,34.226342,-118.058944,1360,1520,3072,2048,0,0.206730


In [145]:
lm_df.to_csv("data/landmarks_processed.csv", index=False)